In [188]:
# import libraries
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

## Data Acquisition
www.baseball-reference.com only allows 20 requests per minute. If we violate this rule, our session will be locked out for one hour.

### Web Crawling

In [189]:
# request website for each year
main_url = 'https://www.baseball-reference.com'
years = ['2019', '2021', '2022']
pages = []
soups = []
for i in range(len(years)):
    url = f'{main_url}/leagues/majors/{years[i]}.shtml'
    page = requests.get(url)
    pages.append(page)
    soups.append(BeautifulSoup(page.content))
    
# three requests

In [190]:
# loop through soups to get the winning team links
pattern = '/teams/[A-Z][A-Z][A-Z]/20[0-9][0-9].shtml'
winning_team_links = []
for k in range(len(soups)):
    hrefs = soups[k].find_all('a', href=True)
    for i in range(len(hrefs)):
        new_str = str(hrefs[i])
        check = re.search(pattern, new_str)
        if check:
            link = new_str[9:30]  
            break
    winning_team_links.append(link)

In [191]:
# request website for each winning team
win_team_pages = []
win_team_soups = []
for i in range(len(winning_team_links)):
    url = f'{main_url}{winning_team_links[i]}'
    page = requests.get(url)
    win_team_pages.append(page)
    win_team_soups.append(BeautifulSoup(page.content))
    
# three requests

In [192]:
# loop through table to get link for each player of each team that won World Series
player_links = []
for k in range(len(win_team_soups)):
    team_pl_links = []
    counter =0
    for this_soup in win_team_soups[k].find_all('td', {"class": "left", "data-stat": "player"}):
        try:
            counter += 1
            link = this_soup.find('a')['href']
            team_pl_links.append(link)
            if counter >= 9:
                break
        except:
            pass
    player_links.append(team_pl_links)

In [193]:
# getting the webpage content for each player
player_pages = []
player_soups = []
for i in range(len(player_links)):
    if i == 1:
        time.sleep(65) # sleep timer because we don't want to exceed 20 requests in a minute, adding 10 seconds for safety
    temp_pages = []
    temp_soups = []
    for j in range(len(player_links[i])):
        url = f'{main_url}{player_links[i][j]}'
        page = requests.get(url)
        temp_pages.append(page)
        temp_soups.append(BeautifulSoup(page.content))
        
    player_pages.append(temp_pages)
    player_soups.append(temp_soups)
    
# 27 requests

In [194]:
# getting links to the gamelogs for each player for the years that the players won the world series
player_game_log_links = []
for i in range(len(player_soups)):
    temp = []
    flag = 0
    for j in range(len(player_soups[i])):
        counter = 0
        flag = 0
        for search in player_soups[i][j].find_all('li', {'data-fade-selector': '#inpage_nav'}):
            for sub_search in search.find_all('ul'):
                counter += 1
                for sub2_search in sub_search.find_all('li'):
                    link = sub2_search.find('a')['href']
                    check = re.search(f'{years[i]}', link)
                    if (check) and (counter == 2 or counter == 3):
                        temp.append(link)
                        flag = 1
                        break
                if flag == 1:
                    break
            if flag == 1:
                break
    player_game_log_links.append(temp)
    
# had to hard-code a fix to one of the links. It was an abnormality.
player_game_log_links[0][2] = '/players/gl.fcgi?id=doziebr01&t=b&year=2019'                        

# print(player_soups[2][2].find_all('li', {'data-fade-selector': '#inpage_nav'}))

### Web Scraping

In [195]:
# create an array of the players names by year
players = []
for i in range(len(player_soups)):
    players_team_array = []
    for j in range(len(player_soups[i])):
        for search in player_soups[i][j].find_all('div', id='meta'):
            name = search.find('h1')
            name = name.text.strip()
        players_team_array.append(name)
    players.append(players_team_array)

In [196]:
# create pandas dataframes for team stats that won the world series
batting_team_stats = []
pitching_team_stats = []
for i in range(len(win_team_pages)):
    [batting, pitching] = pd.read_html(win_team_pages[i].content)
    batting_team_stats.append(batting)
    pitching_team_stats.append(pitching)

# need to delete row index 8 in most (if not all) dataframes for batting. Also delete the last 5 rows.
# delete index row 5, 11, and last 3 rows in pitching. Look at each dataframe before making edits.

In [197]:
# getting the tables for each player that has stats for each game that they played
player_game_stats = []
time.sleep(65) # do not exceed 20 requests in a minute
for i in range(len(player_game_log_links)):
    temp = []
    if i == 2:
        time.sleep(65) # do not exceed 20 requests in a minute
    for j in range(len(player_game_log_links[i])):
        page = requests.get(f'{main_url}{player_game_log_links[i][j]}')
        df = pd.read_html(page.content)
        temp.append(df[4])
    player_game_stats.append(temp)
    
# still need to clean data

### Data Cleaning

### Save Data